In [9]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [2]:
def getSamplingData():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()

        cur.execute("""select samplingfeatureid, samplingfeatureuuid, samplingfeaturecode, samplingfeaturename, 
                       samplingfeaturedescription, elevation_m, elevationdatumcv, samplingfeaturegeotypecv, 
                       samplingfeaturetypecv, st_x(st_astext(featuregeometry)) as longitude, 
                       st_y(st_astext(featuregeometry)) as latitude  from sampling_features""")

        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['samplingfeatureid', 'samplingfeatureuuid', 'samplingfeaturecode', 
                                     'samplingfeaturename','samplingfeaturedescription', 'elevation_m', 
                                     'elevationdatumcv', 'samplingfeaturegeotypecv', 'samplingfeaturetypecv', 
                                     'longitude', 'latitude'])

        for row in rows:
            df = df.append({'samplingfeatureid': row[0], 'samplingfeatureuuid': row[1], 'samplingfeaturecode': row[2], 
                            'samplingfeaturename': row[3],'samplingfeaturedescription': row[4], 'elevation_m': row[5], 
                            'elevationdatumcv': row[6], 'samplingfeaturegeotypecv': row[7], 'samplingfeaturetypecv': row[8], 
                            'longitude': row[9], 'latitude': row[10]}, ignore_index = True)

        return df
                          
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
df = getSamplingData()
df.head()

,samplingfeatureid,samplingfeatureuuid,samplingfeaturecode,samplingfeaturename,samplingfeaturedescription,elevation_m,elevationdatumcv,samplingfeaturegeotypecv,samplingfeaturetypecv,longitude,latitude
0,18848,34b63b0c-0224-4cc6-bd85-96ca1d54acfc,GB50_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB50,None,None,None,Specimen,NaN,NaN
1,10001,7af6f32b-7445-4107-8dcd-1181781befb4,CHA,Charlotte Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-80.741571,35.303831
2,10003,9bf42b1e-d7f8-4b51-b7dd-3c8a5d4c90dd,GRV,Greenville Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-77.484817,35.633604
3,10004,ac72da6e-215a-449d-b54c-9b28985eca3c,RTP,Morrisville Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-78.869148,35.843160
4,10005,9d5339b5-df3e-4266-bc4e-ef16ea82cb64,WIL,Wilmington Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-77.863337,34.220339


In [4]:
outfile = '../ncpfast-db/SampleFeatures.csv'
df.to_csv(outfile, header=True, index=False)

In [5]:
def getSamplingSiteData():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()

        cur.execute("""SELECT s.samplingfeatureid, s.latitude, ST_y(ST_AsText(f.featuregeometry)) AS lat, s.longitude, 
                       ST_x(St_AsText(f.featuregeometry)) AS lon, f.samplingfeaturecode, f.samplingfeaturename,
                       f.samplingfeaturedescription, f.samplingfeaturegeotypecv, f.samplingfeaturetypecv, s.sitetypecv
                       FROM sites s INNER JOIN sampling_features f ON s.samplingfeatureid = f.samplingfeatureid 
                       WHERE f.samplingfeaturecode LIKE 'GB%' OR f.samplingfeaturecode LIKE 'MC%'""")

        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['samplingfeatureid', 'latitude', 'lat', 'longitude', 'lon',
                                     'samplingfeaturecode', 'samplingfeaturename','samplingfeaturedescription', 
                                     'samplingfeaturegeotypecv', 'samplingfeaturetypecv', 'sitetypecv'])

        for row in rows:
            df = df.append({'samplingfeatureid': row[0], 'latitude': row[1], 'lat': row[2], 'longitude': row[3],
                            'lon': row[4], 'samplingfeaturecode': row[5], 'samplingfeaturename': row[6],
                            'samplingfeaturedescription': row[7], 'samplingfeaturegeotypecv': row[8], 
                            'samplingfeaturetypecv': row[9], 'sitetypecv': row[10]}, ignore_index = True)

        return df
                          
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [6]:
df = getSamplingSiteData()
df.head()

,samplingfeatureid,latitude,lat,longitude,lon,samplingfeaturecode,samplingfeaturename,samplingfeaturedescription,samplingfeaturegeotypecv,samplingfeaturetypecv,sitetypecv
0,18279,34.81569,34.81569,-78.83175,-78.83175,GB20,Georgia Branch - GB20,"Testing site in vicinity of Chemours, near Fay...",Point,Site,Stream
1,18280,34.81577,34.81577,-78.83161,-78.83161,GB50,Georgia Branch - GB50,"Testing site in vicinity of Chemours, near Fay...",Point,Site,Stream
2,18281,34.81555,34.81555,-78.83158,-78.83158,GB70,Georgia Branch - GB70,"Testing site in vicinity of Chemours, near Fay...",Point,Site,Stream
3,18282,34.81547,34.81547,-78.83158,-78.83158,GB90,Georgia Branch - GB90,"Testing site in vicinity of Chemours, near Fay...",Point,Site,Stream
4,18283,34.81538,34.81538,-78.83158,-78.83158,GB110,Georgia Branch - GB110,"Testing site in vicinity of Chemours, near Fay...",Point,Site,Stream


In [7]:
outfile = '../ncpfast-db/SiteSampleFeatures.csv'
df.to_csv(outfile, header=True, index=False)

In [8]:
samplingfeatureid = list(df['samplingfeatureid'])

In [10]:
def getFeatureActions(samplingfeatureid):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        rows = []
        for featureid in samplingfeatureid:
            cur.execute("""select featureactionid, actionid, samplingfeatureid 
                           from feature_actions 
                           where samplingfeatureid = %(feature_id)s""",
                       {'feature_id': AsIs(featureid)})
            row = cur.fetchall()
            rows.append(row)
            
        return rows
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [16]:
feature_actions = getFeatureActions(samplingfeatureid)

In [22]:
def getActions(feature_actions):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        rows = []
        for feature_action in feature_actions:
            actionid = feature_action[0][1]
            cur.execute("""select actionid, begin_datetime, end_datetime, methodid
                           from actions 
                           where actionid = %(action_id)s""",
                       {'action_id': AsIs(actionid)})        
            row = cur.fetchall()
            rows.append(row)
            
        return rows
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [23]:
actions = getActions(feature_actions)
actions

[[(148,
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   10003)],
 [(149,
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   10003)],
 [(150,
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   10003)],
 [(151,
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-240, name=None)),
   10003)],
 [(152,
   datetime.datetime(2018, 10, 22, 0, 0, tzinfo=psycopg2.tz.Fixe

In [24]:
len(actions)

30

In [25]:
len(feature_actions)

30

In [26]:
len(samplingfeatureid)

30

In [27]:
feature_actions

[[(216, 148, 18279)],
 [(217, 149, 18280)],
 [(218, 150, 18281)],
 [(219, 151, 18282)],
 [(220, 152, 18283)],
 [(223, 155, 18286)],
 [(228, 160, 18291)],
 [(224, 156, 18287)],
 [(225, 157, 18288)],
 [(222, 154, 18285)],
 [(221, 153, 18284)],
 [(226, 158, 18289)],
 [(227, 159, 18290)],
 [(229, 161, 18292)],
 [(230, 162, 18293)],
 [(231, 163, 18294)],
 [(232, 164, 18295)],
 [(233, 165, 18296)],
 [(234, 166, 18297)],
 [(235, 167, 18298)],
 [(236, 168, 18299)],
 [(237, 169, 18300)],
 [(238, 170, 18301)],
 [(239, 171, 18302)],
 [(240, 172, 18303)],
 [(241, 173, 18304)],
 [(242, 174, 18305)],
 [(243, 175, 18306)],
 [(244, 176, 18307)],
 [(245, 177, 18308)]]